In [1]:
# Importing required libraries

import tensorflow as tf
import numpy as np

from tensorflow.keras import Model, layers
from sklearn.model_selection import train_test_split

2025-11-04 09:00:00.618152: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762243200.639358   19481 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762243200.645601   19481 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762243200.663322   19481 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762243200.663349   19481 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762243200.663352   19481 computation_placer.cc:177] computation placer alr

In [2]:
from tensorflow.keras.datasets import mnist

# Load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to numpy float32
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Normalize data between 0 and 1
x_train, x_test = x_train/255, x_test/255

x_train = np.expand_dims(x_train, axis=-1)
x_test  = np.expand_dims(x_test, axis=-1)


# splitting Data
x_train, valid_data, y_train, valid_labels = train_test_split(
    x_train, y_train,
    test_size = 0.25,
    random_state = 42,
    shuffle=True
)


In [3]:
# Checking data

x_train = tf.pad(x_train, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT", constant_values=0)
valid_data = tf.pad(valid_data, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT", constant_values=0)
x_test = tf.pad(x_test, paddings=[[0, 0], [2, 2], [2, 2], [0, 0]], mode="CONSTANT", constant_values=0)

print(x_train.shape)

(45000, 32, 32, 1)


I0000 00:00:1762243206.565917   19481 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2070 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5


In [4]:
print(x_train[0].shape)

(32, 32, 1)


In [5]:
# Parameters 
total_samples = len(x_train)

train_size = int(0.75 * len(x_train))
num_classes = 10

batch_size = 125
learning_rate = 0.001
epochs = 50
steps_per_epoch = int(np.ceil(total_samples / batch_size))

conv_layer1 = 6
conv_layer2 = 16

In [6]:
# Data batching, splitting and shuffle

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(5000).repeat()


train_data = train_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [7]:
# Defining Model
class LeNet(tf.Module):
    def __init__(self):
        super(LeNet, self).__init__()

        # Convolutional Layer 1
        self.w1 = tf.Variable(tf.random.normal([5, 5, 1, conv_layer1]), name="weight1", trainable=True)
        self.b1 = tf.Variable(tf.zeros([conv_layer1]), name="bias1", trainable=True)

        # Convolutional Layer 2
        self.w2 = tf.Variable(tf.random.normal([5, 5, conv_layer1, conv_layer2]), name="weight2", trainable=True)
        self.b2 = tf.Variable(tf.zeros([conv_layer2]), name="bias2", trainable=True)

        # Fully connected layer 1
        self.w3 = tf.Variable(tf.random.normal([5*5*conv_layer2, 120]), name="weight3", trainable=True)
        self.b3 = tf.Variable(tf.zeros([120]), name="bias3", trainable=True)

        # Fully connected layer 2
        self.w4 = tf.Variable(tf.random.normal([120, 84]), name="weight4", trainable=True)
        self.b4 = tf.Variable(tf.zeros([84]), name="bias4", trainable=True)

        # Output Layer
        self.w5 = tf.Variable(tf.random.normal([84, 10]), name="weight5", trainable=True)
        self.b5 = tf.Variable(tf.zeros([10]), name="bias5", trainable=True)

        
    def conv2d(self, x, filter_W, bias_conv, stride=2, padding="VALID"):
        conv_layer = tf.nn.conv2d(
            x, filter_W,
            strides = [1, stride, stride, 1],
            padding = padding
        )
        conv_layer = tf.nn.bias_add(conv_layer, bias_conv)
        conv_layer = tf.nn.relu(conv_layer)
        return conv_layer

    def maxpool2d(self, x, k=2, s=2):
        return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, s, s, 1], padding="SAME")
    
    def __call__(self, x, is_training=False):
        
        # Layer 1
        conv1 = self.conv2d(x, self.w1, self.b1, 1)
        conv1 = self.maxpool2d(conv1, 2, 2)

        # Layer 2
        conv2 = self.conv2d(conv1, self.w2, self.b2, 1)
        conv2 = self.maxpool2d(conv2, 2, 2)

        # Flatten Layer 
        flatten = tf.reshape(conv2, [-1, self.w3.get_shape().as_list()[0]])

        # Fully connected layer 1
        fc1 = tf.nn.bias_add(tf.matmul(flatten, self.w3), self.b3)
        fc1 = tf.nn.relu(fc1)

        # Fullly connected layer 2
        fc2 = tf.nn.bias_add(tf.matmul(fc1, self.w4), self.b4)
        fc2 = tf.nn.relu(fc2)

        # Output layer
        out = tf.nn.bias_add(tf.matmul(fc2, self.w5), self.b5)

        if not is_training:
            return tf.nn.softmax(out)

        return out

In [8]:
optimizer = tf.optimizers.Adam(learning_rate)
model = LeNet()

In [9]:
def cross_entropy(pred, y_true):
    y_true = tf.cast(y_true, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=pred)
    return tf.reduce_mean(loss)

def accuracy(pred, y_true):
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

In [10]:
def run_optimization(X, Y):
    with tf.GradientTape() as g:
        logits = model(X, is_training=True)
        loss = cross_entropy(logits, Y)

    gradient = g.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradient, model.trainable_variables))

In [11]:
for epoch in range(1, epochs + 1):
    for step, (batch_X, batch_Y) in enumerate(train_data.take(steps_per_epoch), 1):
        run_optimization(batch_X, batch_Y)

    train_pred = model(batch_X, is_training=True)
    train_loss = cross_entropy(train_pred, batch_Y)
    train_acc = accuracy(train_pred, batch_Y)
    valid_pred = model(valid_data, is_training=True)
    valid_loss = cross_entropy(valid_pred, valid_labels)
    valid_acc = accuracy(valid_pred, valid_labels)

    print(f"Epoch {epoch}, train_loss {train_loss}, train_accuracy {train_acc}, validation_loss {valid_loss}, validation_accuracy {valid_acc}")

I0000 00:00:1762243208.212602   19481 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-11-04 09:00:40.493281: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1, train_loss 290.3529357910156, train_accuracy 0.7279999852180481, validation_loss 451.6348876953125, validation_accuracy 0.7337999939918518


2025-11-04 09:01:12.380611: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, train_loss 227.10470581054688, train_accuracy 0.8320000171661377, validation_loss 232.46783447265625, validation_accuracy 0.8129333257675171
Epoch 3, train_loss 64.57658386230469, train_accuracy 0.9039999842643738, validation_loss 157.64808654785156, validation_accuracy 0.8482666611671448


2025-11-04 09:02:21.631719: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 4, train_loss 54.00155258178711, train_accuracy 0.8799999952316284, validation_loss 116.32801818847656, validation_accuracy 0.8668000102043152
Epoch 5, train_loss 58.847015380859375, train_accuracy 0.871999979019165, validation_loss 92.23439025878906, validation_accuracy 0.8814666867256165
Epoch 6, train_loss 62.29881286621094, train_accuracy 0.8640000224113464, validation_loss 79.09906005859375, validation_accuracy 0.8908666372299194
Epoch 7, train_loss 47.945281982421875, train_accuracy 0.9359999895095825, validation_loss 68.80928039550781, validation_accuracy 0.9001333117485046


2025-11-04 09:04:16.067168: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 8, train_loss 45.096988677978516, train_accuracy 0.9120000004768372, validation_loss 58.150508880615234, validation_accuracy 0.9050666689872742
Epoch 9, train_loss 9.306318283081055, train_accuracy 0.9520000219345093, validation_loss 51.075801849365234, validation_accuracy 0.911466658115387
Epoch 10, train_loss 10.138213157653809, train_accuracy 0.9520000219345093, validation_loss 46.30019760131836, validation_accuracy 0.9155333042144775
Epoch 11, train_loss 20.19655990600586, train_accuracy 0.9279999732971191, validation_loss 41.322349548339844, validation_accuracy 0.9177333116531372
Epoch 12, train_loss 22.27643585205078, train_accuracy 0.9440000057220459, validation_loss 38.139732360839844, validation_accuracy 0.9213333129882812
Epoch 13, train_loss 17.441896438598633, train_accuracy 0.9520000219345093, validation_loss 34.837650299072266, validation_accuracy 0.9233333468437195


KeyboardInterrupt: 

In [12]:
test_pred = model(x_test, is_training=True)
test_loss = cross_entropy(test_pred, y_test)
test_acc = accuracy(test_pred, y_test)

print(f"Test loss {test_loss} Test accuracy {test_acc}")

Test loss 34.41279220581055 Test accuracy 0.9289000034332275


In [13]:
print(len(x_test))

10000


In [14]:
test_acc * len(x_test)

<tf.Tensor: shape=(), dtype=float32, numpy=9289.0>